In [531]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Model for NER 
model_name = 'dslim/distilbert-NER'

# Custom keywords
try:
    with open('keyword_matching/directory.pkl', 'rb') as file:
        keywords = pickle.load(file)
except FileNotFoundError or FileExistsError:
    with open('directory.pkl', 'rb') as file:
        keywords = pickle.load(file)

# Hyperparameters
input_size = 512
num_layers = 4          # may require tuning
hidden_size = 256       # may require tuning
num_classes = 97        # 96 different relations plus '0' for no relation
learning_rate = 0.001   # may require tuning
batch_size = 32
num_epochs = 5

In [509]:
import REBRNN
importlib.reload(REBRNN)

dataset='train_annotated'
input_size = 512
batch_size = 32
model_name = 'dslim/distilbert-NER'
custom_keywords = keywords

length = 100

train = CustomDocREDDataset.CustomDocREDDataset(
    dataset='train_annotated',
    input_size=input_size,
    batch_size=batch_size,
    model_name=model_name,
    custom_keywords=custom_keywords,
    device=device,
    shuffle=True,
    length = length*2
)

val = CustomDocREDDataset.CustomDocREDDataset(
    dataset='validation',
    input_size=input_size,
    batch_size=batch_size,
    model_name=model_name,
    custom_keywords=custom_keywords,
    device=device,
    shuffle=True,
    length=length
)

test = CustomDocREDDataset.CustomDocREDDataset(
    dataset='test',
    input_size=input_size,
    batch_size=batch_size,
    model_name=model_name,
    custom_keywords=custom_keywords,
    device=device,
    shuffle=True,
    length=length
)

Starting preprocessing...
0.00% finished
25.00% finished
50.00% finished
75.00% finished
Preprocessing took 0.75 minutes.
Starting preprocessing...
0.00% finished
50.00% finished
Preprocessing took 0.38 minutes.
Starting preprocessing...
0.00% finished
50.00% finished
Preprocessing took 0.40 minutes.


In [535]:
from torch.utils.data import DataLoader, Dataset
import importlib
import CustomDocREDDataset
importlib.reload(CustomDocREDDataset)

train = CustomDocREDDataset.CustomDocREDDataset(
    dataset='train_annotated',
    input_size=input_size,
    model_name=model_name,
    custom_keywords=custom_keywords,
    device=device,
    length = 10
)

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True, collate_fn=CustomDocREDDataset.custom_collate_fn)

Starting preprocessing...
0.00% finished
Preprocessing took 0.04 minutes.


In [536]:
print(train_loader[0])

TypeError: 'DataLoader' object is not subscriptable

In [130]:
custom_loss_fn = REBRNN.CustomLoss(threshold=0.8)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [503]:
'''
To do:
- maybe cleaning text helps? check how that aligns with the triplets
- entity pos still not correct
- sanity check ending position
'''

indices = [#1414, 
           #1939, 
           #2271, 
           2376
           ]

for index in indices:
    sents, vertexSet, labels = train.get_info(train.raw_data.iloc[index])
    print('printing sents manually from here...')
    for i, s in enumerate(sents):
        print(f'{i}: {s}')
        print(s.split(' '))
    pass
    entities = train.extract_entities(sents)
    indexed_entities = train.get_entity_positions(sents, entities, index)
    indexed_triplets = train.make_triplets(vertexSet, labels)
    for t in indexed_triplets:
        print(t)
    for i in indexed_entities:
        print(i)

sent raw:
['The', 'Mascarene', 'martin', 'or', 'Mascarene', 'swallow', '(', 'Phedina', 'borbonica', ')', 'is', 'a', 'passerine', 'bird', 'in', 'the', 'swallow', 'family', 'that', 'breeds', 'in', 'Madagascar', 'and', 'in', 'the', 'Mascarene', 'Islands', '.']
['The', 'nominate', 'subspecies', 'occurs', 'on', 'Mauritius', 'and', 'Réunion', 'and', 'has', 'never', 'been', 'found', 'away', 'from', 'the', 'Mascarene', 'Islands', ',', 'but', 'the', 'smaller', 'Madagascan', 'subspecies', ',', 'P.', '\xa0', 'b.', '\xa0', 'madagascariensis', ',', 'is', 'migratory', 'and', 'has', 'been', 'recorded', 'wintering', 'in', 'East', 'Africa', 'or', 'wandering', 'to', 'other', 'Indian', 'Ocean', 'islands', '.']
['The', 'Mascarene', 'martin', 'is', 'a', 'small', 'swallow', 'that', 'has', 'grey', '-', 'brown', 'underparts', 'becoming', 'white', 'on', 'the', 'throat', 'and', 'lower', 'abdomen', ',', 'dark', 'grey', '-', 'brown', 'upperparts', 'and', 'a', 'slightly', 'forked', 'tail', '.']
['The', 'underparts